In [ ]:
!pip install pandas pymongo transformers

In [2]:
import os
import json
import torch
import re
import ast
import pandas as pd
from pymongo import MongoClient
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from warnings import filterwarnings
filterwarnings(action='ignore')

In [3]:
# Download or Load model

def load_model():

    MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"
    LOCAL_MODEL_DIR = "./model"

    if not os.path.exists(LOCAL_MODEL_DIR) or not os.listdir(LOCAL_MODEL_DIR):
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                                    torch_dtype=torch.float16,
                                                    device_map="auto")
        tokenizer.save_pretrained(LOCAL_MODEL_DIR)
        model.save_pretrained(LOCAL_MODEL_DIR)

    else:
        tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_DIR)
        model = AutoModelForCausalLM.from_pretrained(LOCAL_MODEL_DIR,
                                                    torch_dtype=torch.float16,
                                                    device_map="auto")

    return tokenizer, model

In [4]:
tokenizer, model = load_model()

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
columns = ['ProductID', 'ProductName', 'Category', 'Price', 'Rating', 'ReviewCount', 'Stock', 'Discount', 'Brand', 'LaunchDate']
prompt = f"Convert to MongoDB query: Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand Nike or Sony.The valid column names are {', '.join(columns)}."

inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
result = tokenizer.decode(outputs[0])

query = result.split('.find(')[1].split(')\n\\end{code}')[0].replace('\n','').replace(' ','').split(')`')[0]
query

'{$and:[{Rating:{$lt:4.5}},{ReviewCount:{$gt:200}},{Brand:{$in:["Nike","Sony"]}}]}'

In [7]:
result

'<s> Convert to MongoDB query: Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand Nike or Sony.The valid column names are ProductID, ProductName, Category, Price, Rating, ReviewCount, Stock, Discount, Brand, LaunchDate.\n\n\\begin{code}\ndb.products.find({\n    $and: [\n        {\n            Rating: {\n                $lt: 4.5\n            }\n        },\n        {\n            ReviewCount: {\n                $gt: 200\n            }\n        },\n        {\n            Brand: {\n                $in: [\n                    "Nike",\n                    "Sony"\n                ]\n            }\n        }\n    ]\n})\n\\end{code}\n\nComment: What is the question?\n\nComment: Convert to MongoDB query: Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand Nike or Sony.The valid column names are ProductID, ProductName, Category, Price, Rating, ReviewCount, Stock, Discount, Brand, LaunchDate.\n\n

In [8]:
# CSV to MongoDB

def load_csv_to_mongodb(csv_path, collection):

    df = pd.read_csv(csv_path)
    json_data = json.loads(df.to_json(orient="records"))
    collection.insert_many(json_data)

    return df.columns.tolist()

In [ ]:
csv_file = "/content/sample_data.csv"
db_name = "ProductDB"
collection_name = "ProductCollection"
queries_file = "Queries_generated.txt"

url = "mongodb://localhost:27017/"
client = MongoClient(url)
db = client[db_name]
collection = db[collection_name]

columns = load_csv_to_mongodb(csv_file, collection)
print('Data Loaded Successfully')

Data Loaded Successfully


In [13]:
print(columns)

['ProductID', 'ProductName', 'Category', 'Price', 'Rating', 'ReviewCount', 'Stock', 'Discount', 'Brand', 'LaunchDate']


In [9]:
# Generate MongoDB Query using LLM

def generate_query(prompt, tokenizer, model):

    inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)

    result = tokenizer.decode(outputs[0])
    query = result.split('.find(')[1].split(')\n\\end{code}')[0].replace('\n','').replace(' ','').split(')`')[0]

    return query

In [10]:
def convert_query_to_dict(query_str):

    # Match any alphanumeric key before a colon and wrap it in double quotes
    query_str_with_quotes = re.sub(r'([{\[\s,])([A-Za-z0-9_.$-]+)(:)', r'\1"\2"\3', query_str)

    # Add quotes around alphanumeric string values (but not numeric values or booleans)
    query_str_with_quotes = re.sub(r'(:\s*)([A-Za-z_.$-][A-Za-z0-9_.$-]*)(?=\s*[},\]])', r'\1"\2"', query_str_with_quotes)

    # Convert the string to a Python dictionary
    try:
        return ast.literal_eval(query_str_with_quotes)

    except ValueError as e:
        print(f"Error in converting to dictionary: {e}")
        return None

In [14]:
query = '{$and:[{Rating:{$gt:4.5}},{ReviewCount:{$lt:250}},{$or:[{Brand:"Sony"},{Brand:"HP"}]}]}'
result = convert_query_to_dict(query)
result

{'$and': [{'Rating': {'$gt': 4.5}},
  {'ReviewCount': {'$lt': 250}},
  {'$or': [{'Brand': 'Sony'}, {'Brand': 'HP'}]}]}

In [15]:
for i in collection.find(result):
  print(i)

{'_id': ObjectId('68206115890afb0fe966dc7d'), 'ProductID': 103, 'ProductName': 'Noise Cancelling Headphones', 'Category': 'Electronics', 'Price': 199.99, 'Rating': 4.8, 'ReviewCount': 125, 'Stock': 60, 'Discount': '15%', 'Brand': 'Sony', 'LaunchDate': '05-09-2021'}


In [16]:
pd.DataFrame(collection.find(result))

,_id,ProductID,ProductName,Category,Price,Rating,ReviewCount,Stock,Discount,Brand,LaunchDate
0,68206115890afb0fe966dc7d,103,Noise Cancelling Headphones,Electronics,199.99,4.8,125,60,15%,Sony,05-09-2021


In [11]:
# Execute Query and Present Data

def execute_query_and_present(collection, query_str, save_csv=False, file_name="output.csv"):

    try:
        # Convert string to dictionary (ensure input is safe)
        query = convert_query_to_dict(query_str)
        results = list(collection.find(query))
        if not results:
            print("No data found for the given query.")
            return

        df = pd.DataFrame(results)
        print(df)

        if save_csv:
            df.to_csv(file_name, index=False)
            print(f"Results saved to {file_name}")

    except Exception as e:
        print("Error executing query:", e)

In [ ]:
def main():

    csv_path = '/content/sample_data.csv'
    db_name = 'ProductDB'
    collection_name = 'ProductCollection'
    queries_file = 'Queries_generated.txt'

    # MongoDB Cretentials
    url = "mongodb://localhost:27017/"
    client = MongoClient(url)
    collection = client[db_name][collection_name]

    # Load Data to MongoDB
    # columns = load_csv_to_mongodb(csv_path, collection)

    # Load Model
    # tokenizer, model = load_model()

    print("\nAvailable columns:", columns)

    while True:

        # Get Input Query from User
        user_input = input("Enter your question about the data (or type 'exit' to stop): ")
        if user_input.lower() == "exit":
            break

        # Generate the Query
        prompt = f"Convert to MongoDB query: {user_input}. The valid column names are {', '.join(columns)}."
        query = generate_query(prompt, tokenizer, model)
        print("Generated Query:", query)

        # Save to query log
        with open(queries_file, "a") as f:
            f.write(f"Q: {user_input}\nQuery generated by Model - {query}\n\n")

        # Get input from User
        action = input("Do you want to (d)isplay or (s)ave the output CSV? (d/s): ").strip().lower()
        file_name = input("Enter output file name (e.g., test_case1.csv): ") if action == 's' else None

        # Retrieve Data from MongoDB Database and Save CSV file
        execute_query_and_present(collection, query, save_csv=(action == 's'), file_name=file_name)

In [19]:
# Test_case 1
if __name__ == '__main__':
    main()


Available columns: ['ProductID', 'ProductName', 'Category', 'Price', 'Rating', 'ReviewCount', 'Stock', 'Discount', 'Brand', 'LaunchDate']
Enter your question about the data (or type 'exit' to stop): Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand 'Nike' or 'Sony'.
Generated Query: {$and:[{Rating:{$lt:4.5}},{ReviewCount:{$gt:200}},{Brand:{$in:["Nike","Sony"]}}]}
Do you want to (d)isplay or (s)ave the output CSV? (d/s): s
Enter output file name (e.g., test_case1.csv): test_case1.csv
                        _id  ProductID    ProductName Category  Price  Rating  \
0  68206115890afb0fe966dc7e        104  Running Shoes   Sports  49.99     4.3   

   ReviewCount  Stock Discount Brand  LaunchDate  
0          500    200      20%  Nike  10-02-2022  
Results saved to test_case1.csv
Enter your question about the data (or type 'exit' to stop): exit


In [20]:
# Test_case 2
if __name__ == '__main__':
    main()


Available columns: ['ProductID', 'ProductName', 'Category', 'Price', 'Rating', 'ReviewCount', 'Stock', 'Discount', 'Brand', 'LaunchDate']
Enter your question about the data (or type 'exit' to stop): Which products in the Electronics category have a rating of 4.5 or higher and are in stock?
Generated Query: {$and:[{Category:"Electronics"},{Rating:{$gte:4.5}},{Stock:{$gt:0}}]}
Do you want to (d)isplay or (s)ave the output CSV? (d/s): s
Enter output file name (e.g., test_case1.csv): test_case2.csv
                        _id  ProductID                  ProductName  \
0  68206115890afb0fe966dc7b        101               Wireless Mouse   
1  68206115890afb0fe966dc7c        102              Gaming Keyboard   
2  68206115890afb0fe966dc7d        103  Noise Cancelling Headphones   
3  68206115890afb0fe966dc7f        105                   Smartwatch   
4  68206115890afb0fe966dc83        109                     Smart TV   

      Category   Price  Rating  ReviewCount  Stock Discount     Brand  \

In [21]:
# Test_case 3
if __name__ == '__main__':
    main()


Available columns: ['ProductID', 'ProductName', 'Category', 'Price', 'Rating', 'ReviewCount', 'Stock', 'Discount', 'Brand', 'LaunchDate']
Enter your question about the data (or type 'exit' to stop): List products launched after January 1, 2022, in the Home & Kitchen or Sports categories with a discount of 10% or more, sorted by price in descending order.
Generated Query: {$and:[{$or:[{Category:"Home&Kitchen"},{Category:"Sports"}]},{LaunchDate:{$gte:ISODate("2022-01-01")}},{Discount:{$gte:10}}]}).sort({Price:-1}
Do you want to (d)isplay or (s)ave the output CSV? (d/s): s
Enter output file name (e.g., test_case1.csv): test_case3.csv
Error executing query: unmatched ')' (<unknown>, line 1)
Enter your question about the data (or type 'exit' to stop): exit
